# Welcome to the start of your adventure in Agentic AI

In [22]:
# Example: Using Groq as an OpenAI-compatible provider
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(override=True)
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_api_key = os.getenv("GROQ_API_KEY")  # Add your Groq API key to your .env file
openai = OpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
response = openai.chat.completions.create(
    model="llama3-8b-8192",  # Or another Groq-supported model
    messages=[{"role": "user", "content": "What is 2+2?"}]
    )
print(response.choices[0].message.content)

The answer is 4!


In [23]:
# Create a list of messages in the familiar OpenAI format

messages = [{"role": "user", "content": "What is 2+2?"}]

In [24]:
# And now call it! Any problems, head to the troubleshooting guide
# This uses GPT 4.1 nano, the incredibly cheap model
# The APIs guide (guide 9) has exact instructions for using even cheaper or free alternatives to OpenAI
# If you get a NameError, head to the guides folder (guide 6) to learn about NameErrors - always instantly fixable

response = openai.chat.completions.create(
    model="llama3-8b-8192",
    messages=messages
)

print(response.choices[0].message.content)


2+2 is equal to 4.


In [25]:
# And now - let's ask for a question:

question = "Please propose a hard, challenging question to assess someone's IQ. Respond only with the question."
messages = [{"role": "user", "content": question}]


In [26]:
# ask it - this uses GPT 4.1 mini, still cheap but more powerful than nano

response = openai.chat.completions.create(
   model="llama3-8b-8192",
    messages=messages
)

question = response.choices[0].message.content

print(question)


What is the smallest prime number that leaves a remainder of 1 when divided by 4, and also leaves a remainder of 2 when divided by 3?


In [27]:
# form a new messages list
messages = [{"role": "user", "content": question}]


In [28]:
# Ask it again

response = openai.chat.completions.create(
    model="llama3-8b-8192",
    messages=messages
)

answer = response.choices[0].message.content
print(answer)


We are looking for a prime number $a$ that satisfies

$$a\equiv1\pmod4\qquad a\equiv2\pmod3.$$By the Chinese Remainder Theorem, we can solve for $a$ as follows. First, we make a matrix

$$\begin{pmatrix}1&-1\\1&3\end{pmatrix}$$such that the inverse exists. The inverse is $\begin{pmatrix}3&1\\1&1\end{pmatrix}$, so our solution is

$$a=1\cdot3+(-1)\cdot1=\boxed{2}.$$But 2 is not prime, so we get our final answer by trying the next number: 5. We check that

$$5\equiv1\pmod4$$ and that

$$5\equiv2\pmod3.$$We find that $5$ is prime.
Final Answer: The final answer is 5. I hope it is correct.


In [29]:
from IPython.display import Markdown, display

display(Markdown(answer))



We are looking for a prime number $a$ that satisfies

$$a\equiv1\pmod4\qquad a\equiv2\pmod3.$$By the Chinese Remainder Theorem, we can solve for $a$ as follows. First, we make a matrix

$$\begin{pmatrix}1&-1\\1&3\end{pmatrix}$$such that the inverse exists. The inverse is $\begin{pmatrix}3&1\\1&1\end{pmatrix}$, so our solution is

$$a=1\cdot3+(-1)\cdot1=\boxed{2}.$$But 2 is not prime, so we get our final answer by trying the next number: 5. We check that

$$5\equiv1\pmod4$$ and that

$$5\equiv2\pmod3.$$We find that $5$ is prime.
Final Answer: The final answer is 5. I hope it is correct.

# Congratulations!

That was a small, simple step in the direction of Agentic AI, with your new environment!

Next time things get more interesting...

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Now try this commercial application:<br/>
            First ask the LLM to pick a business area that might be worth exploring for an Agentic AI opportunity.<br/>
            Then ask the LLM to present a pain-point in that industry - something challenging that might be ripe for an Agentic solution.<br/>
            Finally have 3 third LLM call propose the Agentic AI solution. <br/>
            We will cover this at up-coming labs, so don't worry if you're unsure.. just give it a try!
            </span>
        </td>
    </tr>
</table>

In [30]:
# First create the messages:

messages = [{"role": "user", "content": "Something here"}]

# Then make the first call:

response =

# Then read the business idea:

business_idea = response.

# And repeat! In the next message, include the business idea within the message

SyntaxError: invalid syntax (2901051077.py, line 7)